In [ ]:
import libsbml
import importlib
import amici
import amici.plotting
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import scipy.stats

from gm import gm
from gm_Prep import gm_Prep

In [ ]:
# np.random.seed(1337)

In [ ]:
flagD = 1
th = 6

ts = 30
tsup = 0
NSteps = th*3600/ts
NSteps = int(NSteps)

# SBML model we want to import
sbml_file = 'BigModel_FULL_v4.xml'
# Name of the model that will also be the name of the python module
model_name = sbml_file[0:-4] # 'BigModel_byparts_v1'
# Directory to which the generated model code is written
model_output_dir = model_name

sys.path.insert(0, os.path.abspath(model_output_dir))
model_module = importlib.import_module(model_name)
model = model_module.getModel()
solver = model.getSolver() # Create solver instance
solver.setMaxSteps = 1e10
model.setTimepoints(np.linspace(0,ts)) # np.linspace(0, 30) # set timepoints

spdata0 = pd.read_excel('Species_v4.xlsx',header=0,index_col=0)
spdata = np.double(spdata0.values[:,1])
# spdata[np.argwhere(spdata <= 1e-6)] = 0.0

spVolNames = spdata0.values[:,0]
spVols = np.zeros(shape=(774,1))
spVols[np.argwhere(spVolNames=='Cytoplasm')] = 5.2500E-12
spVols[np.argwhere(spVolNames=='Nucleus')] = 1.7500E-12
spVols[np.argwhere(spVolNames=='Mitochondrion')] = 3.6750E-13
spVols[np.argwhere(spVolNames=='Extracellular')] = 5.0000E-05

kGsRead = pd.read_excel('kGeneMod.xlsx',header=0,index_col=0)
gExp_mpc = np.float64(kGsRead.values[:,0])
mExp_mpc = np.float64(kGsRead.values[:,1])
kGin = np.float64(kGsRead.values[:,2])
kGac = np.float64(kGsRead.values[:,3])
kTCleak = np.float64(kGsRead.values[:,4])
kTCmaxs = np.float64(kGsRead.values[:,5])
kTCd = np.float64(kGsRead.values[:,6])
kmRNAs = kGsRead.values[:,7]

# Read-in the activators matrix and assign concentrations of activators
TAsRead = pd.read_csv('TAs.csv',header=0,index_col=0)
TAs0 = np.float64(TAsRead.values)
# Read-in the repressors matrix and assign concentrations of repressors
TRsRead = pd.read_csv('TRs.csv',header=0,index_col=0)
TRs0 = np.float64(TRsRead.values)

In [ ]:
# for count,ele in enumerate(model.getStateIds()): 
#     print(count,ele) 

In [ ]:
genedata, GenePositionMatrix, AllGenesVec, xgac_mpc_D, xgin_mpc_D, xgac_mpc, xgin_mpc, kTCleak2 \
= gm_Prep(flagD, gExp_mpc, mExp_mpc, kGin, kGac, kTCleak, kTCmaxs, kTCd)

In [ ]:
startTime = datetime.now()
Vn = 1.75E-12 # get from model somehow
Vc = 5.25E-12 # get from model somehow

EGFind = 155
INSind = 161
ligConcsEI = [[0.01,0.0],[0.1,0.0],[1.0,0.0],[10.0,0.0],[0.0,0.17],[0.0,1.7],[0.0,17.0],[0.0,1721.0], \
              [0.01,0.17],[0.01,1721.0],[10.0,0.17],[10.0,1721.0]] 

conds = []
tout_all = np.arange(0,th*3600+1,30)
for count,cond in enumerate(ligConcsEI):
    spdata2 = spdata.copy()
    spdata2[EGFind] = cond[0]    
    spdata2[INSind] = cond[1]  
    xoutS_all = np.zeros(shape=(NSteps+1,len(spdata2)))
    xoutS_all[0,:] = spdata2
    for qq in range(NSteps):
        genedata,AllGenesVec = gm2(flagD,ts,genedata,xoutS_all[qq,:],Vn,Vc,kGin,kGac, \
                                   kTCmaxs,kTCleak,kTCd,AllGenesVec,GenePositionMatrix,TAs0,TRs0)
        for ii,kk in enumerate(kmRNAs):
            model.setFixedParameterById(kk,genedata[ii+282]*(1E9/(Vc*6.023E+23)))
        model.setInitialStates(xoutS_all[qq,:])
        rdata = amici.runAmiciSimulation(model, solver)  # Run simulation
        xoutS_all[qq+1,:] = rdata['x'][-1,:]
    conds.append(xoutS_all)
    print(datetime.now() - startTime)

In [ ]:
# # To write to file
# with pd.ExcelWriter('EI_sims_10A_6hr_det.xlsx') as writer:
#     for qq in range(12):
#         condsDF = pd.DataFrame(data=conds[qq],columns=[ele for ele in model.getStateIds()]) 
#         condsDF.to_excel(writer, sheet_name='Sheet_'+str(qq))

In [ ]:
# # To read-in after the simulations are ran and saved 
# conds = []
# for qq in range(12):
#     conds1 = pd.read_excel('EI_sims_10A_6hr_det.xlsx', 'Sheet_'+str(qq))
#     conds1 = conds1.values[:,1::]
#     conds.append(conds1)

In [ ]:
tppERKinds = [150,154,225,226,227,675,717,728,729,732,737,739,741,749,756,757]
ERKvols = spVols[tppERKinds]

tppAktinds = [32,152,696,703,746,748,753]
Aktvols = spVols[tppAktinds]

tpEIF4EBP1ind = [726]
EIF4EBP1vols = spVols[tpEIF4EBP1ind]

In [ ]:
plt.figure(figsize=(7, 4))
plt.plot(tout_all/60.0,(((conds[0][:,tppERKinds])*((ERKvols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[1][:,tppERKinds])*((ERKvols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[2][:,tppERKinds])*((ERKvols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[3][:,tppERKinds])*((ERKvols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.xlabel('Time (min)')
plt.ylabel('ppERK (nM)', multialignment='center')
plt.legend(['E = low','E = med','E = high','E = highest'])
plt.grid(True)
plt.xlim(0, 360)
plt.ylim(0, 120)
plt.xticks(np.arange(0,360,step=60))
plt.savefig('E_ppERK.png')

plt.figure(figsize=(7, 4))
plt.plot(tout_all/60.0,(((conds[4][:,tppERKinds])*((ERKvols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[5][:,tppERKinds])*((ERKvols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[6][:,tppERKinds])*((ERKvols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[7][:,tppERKinds])*((ERKvols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.xlabel('Time (min)')
plt.ylabel('ppERK (nM)', multialignment='center')
plt.legend(['I = low','I = med','I = high','I = highest'])
plt.grid(True)
plt.xlim(0, 360)
plt.ylim(0, 120)
plt.xticks(np.arange(0,360,step=60))
plt.savefig('I_ppERK.png')

plt.figure(figsize=(7, 4))
plt.plot(tout_all/60.0,(((conds[8][:,tppERKinds])*((ERKvols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[9][:,tppERKinds])*((ERKvols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[10][:,tppERKinds])*((ERKvols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[11][:,tppERKinds])*((ERKvols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.xlabel('Time (min)')
plt.ylabel('ppERK (nM)', multialignment='center')
plt.legend(['E-INS = low-low','E-INS = low-highest','E-INS = highest-low','E-INS = highest-highest'])
plt.grid(True)
plt.xlim(0, 360)
plt.ylim(0, 120)
plt.xticks(np.arange(0,360,step=60))
plt.savefig('EI_ppERK.png')

In [ ]:
plt.figure(figsize=(7, 4))
plt.plot(tout_all/60.0,(((conds[0][:,tppAktinds])*((Aktvols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[1][:,tppAktinds])*((Aktvols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[2][:,tppAktinds])*((Aktvols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[3][:,tppAktinds])*((Aktvols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.xlabel('Time (min)')
plt.ylabel('ppAKT (nM)', multialignment='center')
plt.legend(['E = low','E = med','E = high','E = highest'])
plt.grid(True)
plt.xlim(0, 360)
plt.ylim(0, 40)
plt.xticks(np.arange(0,360,step=60))
plt.savefig('E_ppAKT.png')

plt.figure(figsize=(7, 4))
plt.plot(tout_all/60.0,(((conds[4][:,tppAktinds])*((Aktvols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[5][:,tppAktinds])*((Aktvols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[6][:,tppAktinds])*((Aktvols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[7][:,tppAktinds])*((Aktvols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.xlabel('Time (min)')
plt.ylabel('ppAKT (nM)', multialignment='center')
plt.legend(['I = low','I = med','I = high','I = highest'])
plt.grid(True)
plt.xlim(0, 360)
plt.ylim(0, 40)
plt.xticks(np.arange(0,360,step=60))
plt.savefig('I_ppAKT.png')

plt.figure(figsize=(7, 4))
plt.plot(tout_all/60.0,(((conds[8][:,tppAktinds])*((Aktvols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[9][:,tppAktinds])*((Aktvols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[10][:,tppAktinds])*((Aktvols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[11][:,tppAktinds])*((Aktvols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.xlabel('Time (min)')
plt.ylabel('ppAKT (nM)', multialignment='center')
plt.legend(['E-INS = low-low','E-INS = low-highest','E-INS = highest-low','E-INS = highest-highest'])
plt.grid(True)
plt.xlim(0, 360)
plt.ylim(0, 40)
plt.xticks(np.arange(0,360,step=60))
plt.savefig('EI_ppAKT.png')

In [ ]:
plt.figure(figsize=(7, 4))
plt.plot(tout_all/60.0,(((conds[0][:,tpEIF4EBP1ind])*((EIF4EBP1vols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[1][:,tpEIF4EBP1ind])*((EIF4EBP1vols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[2][:,tpEIF4EBP1ind])*((EIF4EBP1vols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[3][:,tpEIF4EBP1ind])*((EIF4EBP1vols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.xlabel('Time (min)')
plt.ylabel('pEIF4EBP1 (nM)', multialignment='center')
plt.legend(['E = low','E = med','E = high','E = highest'])
plt.grid(True)
plt.xlim(0, 360)
plt.ylim(0, 10)
plt.xticks(np.arange(0,360,step=60))
plt.savefig('E_pEIF4EBP1.png')

plt.figure(figsize=(7, 4))
plt.plot(tout_all/60.0,(((conds[4][:,tpEIF4EBP1ind])*((EIF4EBP1vols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[5][:,tpEIF4EBP1ind])*((EIF4EBP1vols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[6][:,tpEIF4EBP1ind])*((EIF4EBP1vols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[7][:,tpEIF4EBP1ind])*((EIF4EBP1vols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.xlabel('Time (min)')
plt.ylabel('pEIF4EBP1 (nM)', multialignment='center')
plt.legend(['I = low','I = med','I = high','I = highest'])
plt.grid(True)
plt.xlim(0, 360)
plt.ylim(0, 10)
plt.xticks(np.arange(0,360,step=60))
plt.savefig('I_pEIF4EBP1.png')

plt.figure(figsize=(7, 4))
plt.plot(tout_all/60.0,(((conds[8][:,tpEIF4EBP1ind])*((EIF4EBP1vols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[9][:,tpEIF4EBP1ind])*((EIF4EBP1vols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[10][:,tpEIF4EBP1ind])*((EIF4EBP1vols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.plot(tout_all/60.0,(((conds[11][:,tpEIF4EBP1ind])*((EIF4EBP1vols/Vc).transpose())).sum(axis=1)),linewidth=4)
plt.xlabel('Time (min)')
plt.ylabel('pEIF4EBP1 (nM)', multialignment='center')
plt.legend(['E-INS = low-low','E-INS = low-highest','E-INS = highest-low','E-INS = highest-highest'])
plt.grid(True)
plt.xlim(0, 360)
plt.ylim(0, 10)
plt.xticks(np.arange(0,360,step=60))
plt.savefig('EI_pEIF4EBP1.png')